# Web API

This notebook is a tutorial of the API used for submitting simulations to our servers.

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pylab as plt

import tidy3d as td

## Setup

Let's set up a simple siulation to get started.

In [2]:
# set up parameters of simulation
dl = 0.05
pml = td.PML()
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 1/fwidth

# create structure
dielectric = td.Medium.from_nk(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]),
    medium=dielectric)

# create source
source = td.VolumeSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time = td.GaussianPulse(
        freq0=freq0,
        fwidth=fwidth),
    polarization='Ex')

# create monitor
monitor = td.FieldMonitor(
    fields=['Ex', 'Ey', 'Ez'],
    center=(0, 0, 0),
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name='field')

# Initialize simulation
sim = td.Simulation(size=sim_size,
                    grid_size=(dl, dl, dl),
                    structures=[square],
                    sources=[source],
                    monitors=[monitor],
                    run_time=run_time,
                    pml_layers=(pml, pml, pml))

## Running simulation manually

For the most control, you can run the simulation through the Tidy3D web API.
Each simulation running on the server is identified by a `task_id`, which must be specified in the web API.
Let's walk through submitting a simulation this way.

In [3]:
import tidy3d.web as web

# upload the simulaition to our servers
task_id = web.upload(sim, task_name='webAPI')

# start the simulation running
web.start(task_id)

# monitor the simulation, dont move on to next line until completed.
web.monitor(task_id)

# download the results and load into a simulation data object for plotting, post processing etc.
sim_data = web.load(task_id, simulation=sim, path='data/sim.hdf5')

[15:58:08] INFO     Uploaded task 'webAPI' with task_id                          ]8;id=569225;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:84
                    '9e2f863f-55e1-4cbb-974b-0159fcdd2804'.

Output()

[15:58:09] status = queued                                                      ]8;id=483169;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:176

[15:58:18] status = preprocess                                                  ]8;id=454113;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:176

[15:58:41] status = running                                                     ]8;id=858879;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:176

...

Output()

[15:59:07] INFO     downloading file "tidy3d.log" to "data/tidy3d.log"          ]8;id=962111;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393

Output()

           INFO     exporting SimulationData to data/sim.hdf5                   ]8;id=166927;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:266

           INFO     loading SimulationData from data/sim.hdf5                   ]8;id=232121;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:302

While we broke down each of the individual steps above, one can also perform the entire process in one line by calling the web.run() function as follows.

```python
sim_data = web.run(sim, task_name='webAPI', path='data/sim.hdf5')
```

(We wont run it again in this notebook to save time).

Sometimes this is more convenient, but other times it can be helpful to have the steps broken down one by one, for example if the simulation is long, you may want to `web.start()` and then exit the session and load the results at a later time using `web.load`.


## Job Container

The `web.Job` interface provides a more convenient way to manage single simuations, mainly because it eliminates the need for keeping track of the `task_id` and original `Simulation`.

While `Job` has methods with names identical to the web API functions above, which give some more granular control, it is often most convenient to call `.run()` so we will do that now.

In [4]:
# set the logging level to warning to reduce downloading messages.
td.set_logging_level('warning')

# initializes job, puts task on server (but doesnt run it)
job = web.Job(simulation=sim, task_name='job')

# start job, monitor, and load results
sim_data = job.run(path='data/sim.hdf5')

[15:59:08] INFO     Uploaded task 'job' with task_id                             ]8;id=583987;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:84
                    '4902fb08-2c88-48a4-8b0d-2cb0bff1fe80'.

Output()

[15:59:18] status = preprocess                                               ]8;id=990685;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:118

[15:59:41] status = running                                                  ]8;id=413510;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:118

[16:00:01] status = visualize                                                ]8;id=683393;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/tidy3d/tidy3d/web/container.py\container.py]8;;\:118

...

Output()

           INFO     downloading file "tidy3d.log" to "data/tidy3d.log"          ]8;id=407423;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393

Output()

[16:00:07] INFO     exporting SimulationData to data/sim.hdf5                   ]8;id=501631;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:266

           INFO     loading SimulationData from data/sim.hdf5                   ]8;id=70422;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:302

Another convenient thing about `Job` objects is that they can be saved and loaded just like other Tidy3d components.

This is convenient if you want to save and load up the results of a job that has already finished, without needing to know the `task_id`.

In [5]:
# saves the job metadata to a single file
job.to_file('data/job.json')

# can exit session, break here, or continue in new session.

# load the job metadata from file
job_loaded = web.Job.from_file('data/job.json')

# download the data from the server and load it into a SimulationData object.
sim_data = job_loaded.load(path='data/sim.hdf5')

           INFO     downloading file "simulation.json" to                       ]8;id=645924;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393
                    "data/simulation.json"

Output()

           INFO     downloading file "monitor_data.hdf5" to                     ]8;id=549618;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393
                    "data/monitor_data.hdf5"

Output()

[16:00:08] INFO     downloading file "tidy3d.log" to "data/tidy3d.log"          ]8;id=15306;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393

Output()

           INFO     exporting SimulationData to data/sim.hdf5                   ]8;id=288582;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:266

           INFO     loading SimulationData from data/sim.hdf5                   ]8;id=849617;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:302

## Batch Processing

Commonly one needs to submit a batch of Simulations.
One way to approach this using the web API is to upload, start, monitor, and load a series of tasks one by one, but this is clumsy and you can lose your data if the session gets interrupted.

A better way is to use the built-in `Batch()` object.
The batch object is like a `Job()`, but stores task metadata for a series of simulations.

In [6]:
# make a dictionary of  {task name : simulation} for demonstration
sims = {f'sim_{i}': sim for i in range(3)}

# initialize a batch and run them all
batch = web.Batch(simulations=sims)

# run the batch and store all of the data in the `data/` dir.
batch_results = batch.run(path_dir='data')

[16:00:09] INFO     Uploaded task 'sim_0' with task_id                           ]8;id=699856;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:84
                    '2b6d60de-42a0-4f92-bd97-ebf940735793'.

[16:00:10] INFO     Uploaded task 'sim_1' with task_id                           ]8;id=570355;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:84
                    '0d4cb200-7770-4e64-a280-60ef373424a2'.

           INFO     Uploaded task 'sim_2' with task_id                           ]8;id=450269;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:84
                    '06f94971-b1c4-4cdc-b343-9266b5653c80'.

Output()

[16:02:15] INFO     downloading file "simulation.json" to                       ]8;id=938647;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393
                    "data/simulation.json"

...

[16:02:18] INFO     downloading file "monitor_data.hdf5" to                     ]8;id=931353;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393
                    "data/monitor_data.hdf5"

Output()

           INFO     downloading file "tidy3d.log" to "data/tidy3d.log"          ]8;id=60697;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:393

Output()

           INFO     exporting SimulationData to data/sim_2.hdf5                 ]8;id=345700;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/notebooks/../../tidy3d/tidy3d/web/webapi.py\webapi.py]8;;\:266

When the batch is completed, the output is not a dictionary of `SimulationData` but rather a python generator that can be looped through once to get the simulation data one by one.

This was chosen to reduce the memory strain from loading all `SimulationData` objects at once.

Alternatively, the batch can be looped through (several times) using the `.items()` method, similar to a dictionary.

In [15]:
# grab the sum of intensities in the simulation one by one (to save memory)
intensities = {}
for task_name, sim_data in batch.items(path_dir='data'):
    field_data = sim_data.at_centers('field').sel(f=freq0)
    intensity = abs(field_data.Ex)**2 + abs(field_data.Ey)**2 + abs(field_data.Ez)**2
    sum_intensity = float(intensity.sum(('x', 'y')).values)
    intensities[task_name] = sum_intensity

print(intensities)

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

{'sim_0': 7.288716652138652e-28,
 'sim_1': 7.288716652138652e-28,
 'sim_2': 7.288716652138652e-28}
